Library Imports

In [2]:
import pandas as pd
import numpy as np
import math
import xlsxwriter
import requests

In [3]:

stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 66915558,
 'calculationPrice': 'close',
 'change': -2.72,
 'changePercent': -0.0156,
 'close': 171.92,
 'closeSource': 'ifflaoic',
 'closeTime': 1675662230660,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 177.74,
 'delayedPriceTime': 1685685101392,
 'extendedChange': -0.8,
 'extendedChangePercent': -0.00459,
 'extendedPrice': 172.32,
 'extendedPriceTime': 1725574907385,
 'high': 175.3,
 'highSource': 'Cloes',
 'highTime': 1713397362163,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 175.84,
 'iexCloseTime': 1724825779930,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 179.765,
 'iexOpenTime': 1735157521932,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1675012214401,
 'latestPrice': 176.45,
 'latestSource': 'Close',
 'latestTime': 'August 19, 2022',
 'latestUpdate': 1674926268147,
 'latestVolume': 73071095,
 'low': 171.92,
 'lowSource

In [5]:
data['latestPrice']
data['marketCap']

2866799473655

Adding stocks data to pandas dataframe

In [6]:

my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to buy ']
final_dataframe = pd.DataFrame (columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy


In [7]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

C:\Users\adeen\AppData\Local\Temp\ipykernel_6804\2737606595.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,AAPL,176.45,2866799473655,N/A


Looping Through The Tickers in Our List of Stocks

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,138.95,41649471175,N/A
1,AAL,14.78,9294871197,N/A
2,AAP,208.7,12768052756,N/A
3,AAPL,175.08,2847723613743,N/A
4,ABBV,144.78,253881469953,N/A
5,ABC,155.8,32599536384,N/A
6,ABMD,277.4,12680475069,N/A
7,ABT,114.75,198019011006,N/A
8,ACN,323.16,211709848691,N/A
9,ADBE,442.699,202347497924,N/A


Using batch api calls to improve performance
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks

In [10]:
def chunks(lst,n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    


In [ ]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,139.02,42775641901,N/A
1,AAL,14.18,9549005952,N/A
2,AAP,208.52,13074203490,N/A
3,AAPL,175.39,2890460183616,N/A
4,ABBV,143.65,260571796115,N/A
...,...,...,...,...
130,DG,258.32,59847874456,N/A
131,DGX,137.87,16091034915,N/A
132,DHI,75.07,26184003959,N/A
133,DHR,298.13,220812630306,N/A


In [13]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
positon_size = val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of shares to buy:'] = math.floor(position_size / final_dataframe['Price'][i])

# number_of_apple_shares = positon_size/500
# print(math.floor(number_of_apple_shares))

In [ ]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,142.01,42499842410,N/A
1,AAL,14.42,9360268330,N/A
2,AAP,215.65,13104075693,N/A
3,AAPL,179.35,2773903416401,N/A
4,ABBV,147.58,252392538504,N/A
5,ABC,155.4,31944423338,N/A
6,ABMD,279.21,12773445064,N/A
7,ABT,114.29,200139009409,N/A
8,ACN,326.64,216604671392,N/A
9,ADBE,430.239,205359018392,N/A


Formatting our Excel Output  
"intitilizing our Xlxswriter object"

In [ ]:
writer = pd.ExcelWriter('recommended trades.xlxs', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

Creating the format required for .xlxs file

-format includes colors, fontstyles, symbols

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
                'font_color': font_color,
                'bg_color': background_color,
                'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
                'num_format': '$0.00',
                'font_color': font_color,
                'bg_color': background_color,
                'border': 1
    }
)

integer_format = writer.book.add_format(
    {
                'num_format': '0',
                'font_color': font_color,
                'bg_color': background_color,
                'border': 1
    }
)

Applying the Formats to the Columns of Our .xlsx File
We can use the set_column method applied to the writer.sheets['Recommended Trades'] object to apply formats to specific columns of our spreadsheets.

Here's an example:

writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )

In [ ]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)
writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself".

Let's simplify this by putting it in 2 loops:

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

Saving Our Excel Output
Saving our Excel file is very easy:

In [ ]:
writer.save()